# Agents From Scratch

Link: [Blog post](https://www.newsletter.swirlai.com/p/building-ai-agents-from-scratch-part)

In [1]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Literal, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl

# Visualization
import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from rich.table import Table
from rich import box
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)

console = Console(theme=custom_theme)

In [3]:
from dataclasses import dataclass
import inspect
import os
from typing import _GenericAlias, Callable, get_type_hints
from urllib import request as urllib_request

import openai

### Tool Creation Utilities

In [4]:
@dataclass
class Tool:
    """A class representing a tool with a name, description, function, and parameters.

    Parameters
    ----------
    name : str
        The name of the tool
    description : str
        A description of what the tool does
    func : Callable[..., str]
        The function that implements the tool's functionality
    parameters : dict[str, dict[str, str]]
        A dictionary mapping parameter names to their descriptions and types
    """

    name: str
    description: str
    func: Callable[..., str]
    parameters: dict[str, dict[str, str]]

    def __call__(self, *args: Any, **kwds: Any) -> str:
        """Execute the tool's function with the given arguments.

        Parameters
        ----------
        *args : Any
            Positional arguments to pass to the function
        **kwds : Any
            Keyword arguments to pass to the function

        Returns
        -------
        str
            The result of executing the function
        """
        return self.func(*args, **kwds)


def parse_docstring_params(docstring: str) -> dict[str, str]:
    """Parse parameters section from a NumPy-style docstring.

    Parameters
    ----------
    docstring : str
        The docstring to parse

    Returns
    -------
    dict[str, str]
        A dictionary mapping parameter names to their descriptions
    """
    if not docstring:
        return {}

    params: dict[str, str] = {}
    lines: list[str] = docstring.split("\n")
    in_params: bool = False
    current_param: str | None = None

    for line in lines:
        line = line.strip()
        if line.startswith("Parameters:"):
            in_params = True
        elif in_params:
            if line.startswith("-") or line.startswith("*"):
                current_param = line.lstrip("- *").split(":")[0].strip()
                params[current_param] = line.lstrip("- *").split(":")[1].strip()
            elif current_param and line:
                params[current_param] += " " + line
            elif not line:
                in_params = False

    return params


def get_type_description(type_hint: Any) -> str:
    """Get a string description of a type hint.

    Parameters
    ----------
    type_hint : Any
        The type hint to describe

    Returns
    -------
    str
        A human-readable description of the type
    """
    if isinstance(type_hint, _GenericAlias):
        if type_hint._name == "Literal":
            return f"one of {type_hint.__args__}"
    return type_hint.__name__

In [5]:
# Updated version
def parse_docstring_params(docstring: str) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    This function extracts parameter descriptions from a NumPy-style docstring
    and returns them as a dictionary mapping parameter names to their descriptions.

    Parameters
    ----------
    docstring : str
        The docstring to parse, expected to be in NumPy format.

    Returns
    -------
    dict[str, str]
        A dictionary where keys are parameter names and values are their descriptions.
        Returns an empty dict if the docstring is empty or has no parameters.
    """
    if not docstring:
        return {}

    params: dict[str, str] = {}
    lines: list[str] = docstring.split("\n")
    in_params: bool = False
    current_param: str | None = None

    for line in lines:
        line = line.strip()
        if line.startswith("Parameters"):
            in_params = True
            continue
        elif in_params:
            if (
                line.startswith("Returns")
                or line.startswith("Raises")
                or line.startswith("Notes")
                or line.startswith("Examples")
            ):
                break
            if line:
                if ":" in line:
                    current_param, description = map(str.strip, line.split(":", 1))
                    # Remove leading "-", " " or "*" from the line
                    current_param = current_param.strip("-* ")
                    params[current_param] = description
                elif current_param:
                    params[current_param] += " " + line
            else:
                in_params = False

    return params

In [6]:
# Examples
# 1: Regular
def my_func1(param1: str, param2: int) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    Parameters:
    - param1: This is the first parameter.
    - param2: This is the second parameter.

    Returns:
    - A dictionary with parameter names as keys and their descriptions as values.
    """


# 2: NumPy Style
def my_func2(param1: str, param2: int) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    Parameters
    ----------
    param1 : str
        This is the first parameter.
    param2 : int
        This is the second parameter.

    Returns
    -------
    dict[str, str]
        A dictionary with parameter names as keys and their descriptions as values.
    """


console.print(parse_docstring_params(my_func1.__doc__))
console.print(parse_docstring_params(my_func2.__doc__))

{'param1': 'This is the first parameter.', 'param2': 'This is the second parameter.'}

{'param1': 'str This is the first parameter.', 'param2': 'int This is the second parameter.'}

In [7]:
type_hints_dict: dict[str, Any] = get_type_hints(my_func2)
print(type_hints_dict)

print(get_type_description(type_hints_dict.get("param1")))

{'param1': <class 'str'>, 'param2': <class 'int'>, 'return': dict[str, str]}
str


<br>

### Tool Creation Decorator

In [8]:
console.print(inspect.getdoc(my_func1))

print("===== OR =====\n\n")

# OR
print(my_func1.__doc__)

Parse docstring parameters into a dictionary.

Parameters:
- param1: This is the first parameter.
- param2: This is the second parameter.

Returns:
- A dictionary with parameter names as keys and their descriptions as values.

===== OR =====


Parse docstring parameters into a dictionary.

    Parameters:
    - param1: This is the first parameter.
    - param2: This is the second parameter.

    Returns:
    - A dictionary with parameter names as keys and their descriptions as values.
    


In [9]:
print(inspect.signature(my_func2))
print(inspect.signature(my_func2).parameters.items())

(param1: str, param2: int) -> dict[str, str]
odict_items([('param1', <Parameter "param1: str">), ('param2', <Parameter "param2: int">)])


In [10]:
def tool(name: str | None = None) -> str:
    """Decorator function to create a Tool object from a function.

    Parameters
    ----------
    name : str | None, optional
        Custom name for the tool. If None, uses the function name, by default None

    Returns
    -------
    Callable
        Decorator function that creates a Tool object
    """

    def decorator(func: Callable[..., str]) -> Tool:
        """Inner decorator function that processes the decorated function.

        Parameters
        ----------
        func : Callable[..., str]
            Function to be converted into a Tool object

        Returns
        -------
        Tool
            Tool object created from the decorated function
        """
        tool_name: str = name or func.__name__
        description: str = inspect.getdoc(func) or "No description provided."

        type_hints: dict[str, Any] = get_type_hints(func)
        param_docs: dict[str, str] = parse_docstring_params(description)
        sig: inspect.Signature = inspect.signature(func)
        params: dict[str, Any] = {}

        for param_name, _ in sig.parameters.items():
            params[param_name] = {
                "type": get_type_description(type_hints.get(param_name)),
                "description": param_docs.get(param_name, "No description provided."),
            }
        return Tool(
            name=tool_name,
            description=description,
            func=func,
            parameters=params,
        )

    return decorator

### Create Currency Convertion Tool

In [11]:
@tool()
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert an amount from one currency to another using exchange rates.

    Parameters
    ----------
    amount : float
        The amount of money to convert
    from_currency : str
        The source currency code (e.g., 'USD', 'EUR', NGN, etc)
    to_currency : str
        The target currency code (e.g., 'USD', 'EUR', NGN, etc)

    Returns
    -------
    str
        A formatted string containing the conversion result or error message
    """
    try:
        url: str = f"https://api.exchangerate-api.com/v4/latest/{from_currency.upper()}"
        with urllib_request.urlopen(url) as response:
            data: dict[str, dict[str, float] | str] = json.loads(response.read())

        if "rates" not in data:
            return "Error: Could not retrieve exchange rates."

        rate: float = data["rates"].get(to_currency.upper())  # type: ignore
        if not rate:
            return f"Error: Currency conversion not supported for {to_currency}."

        converted: float = amount * rate
        return f"{amount} {from_currency} is equal to {converted:,} {to_currency}"

    except Exception as e:
        return f"Conversion error: {str(e)}"

In [12]:
convert_currency(amount=1000, from_currency="USD", to_currency="NGN")

'1000 USD is equal to 1,538,530.0 NGN'

In [13]:
import requests

from_currency: str = "USD"
url: str = f"https://api.exchangerate-api.com/v4/latest/{from_currency.upper()}"

resp = requests.get(url)
resp.json().get("rates")

{'USD': 1,
 'AED': 3.67,
 'AFN': 70.3,
 'ALL': 94.71,
 'AMD': 397.62,
 'ANG': 1.79,
 'AOA': 924.07,
 'ARS': 1030.5,
 'AUD': 1.61,
 'AWG': 1.79,
 'AZN': 1.7,
 'BAM': 1.88,
 'BBD': 2,
 'BDT': 119.49,
 'BGN': 1.88,
 'BHD': 0.376,
 'BIF': 2931.36,
 'BMD': 1,
 'BND': 1.36,
 'BOB': 6.93,
 'BRL': 6.19,
 'BSD': 1,
 'BTN': 85.47,
 'BWP': 13.92,
 'BYN': 3.39,
 'BZD': 2,
 'CAD': 1.44,
 'CDF': 2842.86,
 'CHF': 0.901,
 'CLP': 990.44,
 'CNY': 7.3,
 'COP': 4407.04,
 'CRC': 506.64,
 'CUP': 24,
 'CVE': 105.82,
 'CZK': 24.17,
 'DJF': 177.72,
 'DKK': 7.16,
 'DOP': 60.82,
 'DZD': 135.37,
 'EGP': 50.87,
 'ERN': 15,
 'ETB': 125.94,
 'EUR': 0.96,
 'FJD': 2.32,
 'FKP': 0.796,
 'FOK': 7.16,
 'GBP': 0.796,
 'GEL': 2.8,
 'GGP': 0.796,
 'GHS': 15.12,
 'GIP': 0.796,
 'GMD': 72.5,
 'GNF': 8622.48,
 'GTQ': 7.7,
 'GYD': 209.15,
 'HKD': 7.76,
 'HNL': 25.4,
 'HRK': 7.23,
 'HTG': 130.68,
 'HUF': 394.11,
 'IDR': 16222.66,
 'ILS': 3.69,
 'IMP': 0.796,
 'INR': 85.47,
 'IQD': 1310.55,
 'IRR': 42075.76,
 'ISK': 139.2,
 'JEP'

In [55]:
@tool()
def perform_math(
    first_number: float, second_number: float, operation: Literal["add", "subtract"]
) -> str:
    """Perform basic arithmetic operations (addition or subtraction) on two numbers.

    Parameters
    ----------
    first_number : float
        The first number in the operation
    second_number : float
        The second number in the operation
    operation : Literal["add", "subtract"]
        The type of operation to perform ('add' or 'subtract')

    Returns
    -------
    str
        A formatted string containing the calculation result or error message

    Examples
    --------
    >>> perform_math(5.0, 3.0, "add")
    '5.0 + 3.0 = 8.0'
    >>> perform_math(10.0, 4.0, "subtract")
    '10.0 - 4.0 = 6.0'
    """
    operation: str = operation.lower()
    if operation not in ["add", "subtract"]:
        return "Error: Operation must be either 'add' or 'subtract'"

    try:
        if isinstance(first_number, str) and isinstance(second_number, str):
            first_number: float = float(first_number.replace(",", "").strip())
            second_number: float = float(second_number.replace(",", "").strip())

        if operation == "add":
            result: float = first_number + second_number
            return f"{first_number} + {second_number} = {result:,}"
        else:
            result = first_number - second_number
            return f"{first_number} - {second_number} = {result:,}"

    except Exception as e:
        return f"Calculation error: {str(e)}"


import time


@tool()
def get_current_time(city: str, continent: str) -> str:
    """Get the current time for a given city and continent.

    Parameters
    ----------
    city : str
        Name of the city to get time for. Can be single or multi-word city name.
    continent : str
        Name of the continent where the city is located.

    Returns
    -------
    str
        A formatted string containing the current date and time for the specified location.
        In case of error, returns an error message.

    Examples
    --------
    >>> get_current_time("New York", "America")
    'The current time in New York, [America] is date: 2024-01-20 and time: 14:30:00'
    """
    city_list: list[str] = city.strip().title().split(" ")
    if len(city_list) == 2:
        city_: str = f"{city_list[0]}_{city_list[1]}"
    else:
        city_: str = city.strip().title()
    continent: str = continent.title()
    try:
        url: str = (
            f"https://timeapi.io/api/time/current/zone?timeZone={continent}%2F{city_}"
        )
        response: requests.Response = requests.get(url)
        response.raise_for_status()
        response_json: dict[str, Any] = response.json()
        return (
            f"The current time in {city}, [{continent}] is "
            f"date: {response_json['date']} and time: {response_json['time']}"
        )
    except requests.exceptions.HTTPError as e:
        return f"Error getting current time: \n{e}"

In [15]:
perform_math(first_number=10, second_number=5, operation="add")

'10 + 5 = 15'

### Load Environment Variables

In [16]:
from pydantic_settings import BaseSettings, SettingsConfigDict


class Settings(BaseSettings):
    """Settings for the application."""

    OPENAI_API_KEY: str
    model_config = SettingsConfigDict(env_file=".env", env_file_encoding="utf-8")


settings = Settings()

### Agent Class

In [56]:
class Agent:
    """A class that manages AI tools and executes queries using OpenAI's API.

    This class provides functionality to add tools, manage them, and execute
    user queries by either using the tools or providing direct responses.

    Attributes
    ----------
    client : openai.OpenAI
        The OpenAI client instance for making API calls
    tools : dict[str, Tool]
        Dictionary mapping tool names to Tool instances
    """

    def __init__(self) -> None:
        """Initialize the Agent with OpenAI client and empty tools dictionary.

        The initialization sets up the OpenAI client with the API key and creates
        an empty dictionary to store tools.

        Attributes
        ----------
        client : openai.OpenAI
            The OpenAI client instance for API calls
        tools : dict[str, Tool]
            Dictionary storing tool instances with their names as keys
        """
        self.client = openai.OpenAI(api_key=settings.OPENAI_API_KEY)
        self.tools: dict[str, Tool] = {}

    def add_tool(self, tool: Tool) -> None:
        """Add a tool to the agent's available tools.

        Parameters
        ----------
        tool : Tool
            The tool instance to add to the agent's toolkit

        Returns
        -------
        None
        """
        self.tools[tool.name] = tool

    def add_multiple_tools(self, tools: list[Tool]) -> None:
        """Add multiple tools to the agent's available tools.

        Parameters
        ----------
        tools : list[Tool]
            List of Tool instances to add to the agent's toolkit

        Returns
        -------
        None
        """
        for tool in tools:
            self.add_tool(tool)

    def get_available_tools(self) -> list[str]:
        """Get a list of available tools with their descriptions.

        Returns
        -------
        list[str]
            List of strings containing tool names and descriptions
        """
        return [f"{tool.name}: {tool.description}" for tool in self.tools.values()]

    def use_tool(self, tool_name: str, **kwargs: Any) -> Any:
        """Execute a specific tool with given arguments.

        Parameters
        ----------
        tool_name : str
            Name of the tool to execute
        **kwargs : Any
            Keyword arguments to pass to the tool

        Returns
        -------
        Any
            Result of the tool execution

        Raises
        ------
        ValueError
            If the specified tool is not found
        """
        if tool_name not in self.tools:
            raise ValueError(
                f"Tool '{tool_name}' not found. Available tools: {list(self.tools.keys())}"
            )

        tool = self.tools[tool_name]
        return tool.func(**kwargs)

    def create_system_prompt(self) -> str:
        """Create the system prompt for the AI assistant.

        Returns
        -------
        str
            Formatted system prompt containing tools and instructions
        """
        tools_json: dict[str, Any] = {
            "role": "AI Assistant",
            "capabilities": [
                "Using provided tools to help users when needed.",
                "Responding directly to user queries when no tools are needed.",
                "Combining multiple tools when needed to solve complex tasks",
                "Planning efficient tool usage sequences.",
            ],
            "instructions": [
                "Use tools only when they are needed.",
                "If a query can be answered without tools, respond directly.",
                "Combine multiple tools when needed to solve complex tasks",
                "When tool are needed, plan their usage efficiently to minimize tools calls.",
            ],
            "tools": [
                {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": {
                        name: {
                            "type": info["type"],
                            "description": info["description"],
                        }
                        for name, info in tool.parameters.items()
                    },
                }
                for tool in self.tools.values()
            ],
            "response_format": {
                "type": "json",
                "schema": {
                    "requires_tools": {
                        "type": "boolean",
                        "description": "whether this query requires the use of tools.",
                    },
                    "direct_response": {
                        "type": "string",
                        "description": "the response to the query if no tools are needed.",
                        "optional": True,
                    },
                    "thought": {
                        "type": "string",
                        "description": "the thought process of the AI assistant (when tools are needed).",
                        "optional": True,
                    },
                    "plan": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "steps to solve the task (when tools are needed).",
                        "optional": True,
                    },
                    "tool_calls": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "tool": {
                                    "type": "string",
                                    "description": "the name of the tool to call.",
                                },
                                "args": {
                                    "type": "string",
                                    "description": "the parameters for the tool.",
                                },
                            },
                        },
                        "description": "the tools to be called (when tools are needed).",
                        "optional": True,
                    },
                },
                # Optional:
                "examples": [
                    {
                        "query": "Convert 100 USD to EUR",
                        "response": {
                            "requires_tools": True,
                            "thought": "I need to use the currency converter tool to convert USD to EUR.",
                            "plan": [
                                "Use the convert_currency tool to convert 100 USD to EUR",
                                "Return the result of the conversion",
                            ],
                            "tool_calls": [
                                {
                                    "tool": "convert_currency",
                                    "args": {
                                        "amount": 100,
                                        "from_currency": "USD",
                                        "to_currency": "EUR",
                                    },
                                }
                            ],
                        },
                    },
                    {
                        "query": "Convert 50 USD to NGN",
                        "response": {
                            "requires_tools": True,
                            "thought": "I need to use the currency converter tool to convert USD to NGN.",
                            "plan": [
                                "Use the convert_currency tool to convert 50 USD to NGN",
                                "Return the result of the conversion",
                            ],
                            "tool_calls": [
                                {
                                    "tool": "convert_currency",
                                    "args": {
                                        "amount": 50,
                                        "from_currency": "USD",
                                        "to_currency": "NGN",
                                    },
                                }
                            ],
                        },
                    },
                    {
                        "query": "What currency does Nigeria use?",
                        "response": {
                            "requires_tools": False,
                            "direct_response": "Nigeria uses the Nigerian Naira (NGN) as its official currency. "
                            "This is common knowledge that doesn't require using the currency conversion tool.",
                        },
                    },
                ],
            },
        }
        return f"""You're an AI assistant that helps users by providing direct answers or using tools when necessary.
    Configuration, instructions, and available tools are provided in JSON format below:

    {json.dumps(tools_json, indent=2)}

    Always respond with a JSON object following the response_format schema above. 
    Remember to use tools only when they are actually needed for the task."""

    def plan(self, user_query: str) -> dict[str, Any]:
        """Create an execution plan for the user query.

        Parameters
        ----------
        user_query : str
            The query from the user

        Returns
        -------
        dict[str, Any]
            A dictionary containing the execution plan

        Raises
        ------
        ValueError
            If the response cannot be parsed as JSON
        """
        messages = [
            {"role": "system", "content": self.create_system_prompt()},
            {"role": "user", "content": user_query},
        ]
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0,
        )
        try:
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            raise ValueError("Failed to parse response as JSON.")

    def execute(self, user_query: str) -> str:
        """Execute the user query using available tools or direct response.

        Parameters
        ----------
        user_query : str
            The query from the user

        Returns
        -------
        str
            The execution result or error message
        """
        try:
            plan = self.plan(user_query)
            if not plan.get("requires_tools", True):
                return plan["direct_response"]

            # Execute each tool sequence
            results: list[str] = []
            for tool_call in plan["tool_calls"]:
                tool_name = tool_call["tool"]
                tool_args = tool_call["args"]
                result = self.use_tool(tool_name, **tool_args)
                results.append(result)

            return (
                f"Thought: {plan['thought']}"
                f"\nPlan: {'. '.join(plan['plan'])}"
                f"\nResults: {'. '.join(results)}"
            )
        except Exception as e:
            return f"Error execting plan: {str(e)}"

### Create and run the Agent

In [58]:
agent = Agent()
agent.add_multiple_tools([convert_currency, perform_math, get_current_time])


query_list: list[str] = [
    "Convert 100 USD to NGN",
    "What is the the time in Lagos, Africa?",
    "What's the population of Nigeria?",
    "What is the time in New York (America) right now?",
    "Subtract 100 from 1000",
]

response_dict: dict[str, Any] = {}
for query in query_list:
    response = agent.execute(query)
    response_dict[query] = response

    console.print(f"Query: {query}\nResponse: {response}\n")

Query: Convert 100 USD to NGN
Response: Thought: I need to use the currency converter tool to convert USD to NGN.
Plan: Use the convert_currency tool to convert 100 USD to NGN. Return the result of the conversion
Results: 100 USD is equal to 153,853.0 NGN

Query: What is the the time in Lagos, Africa?
Response: Thought: I need to use the get_current_time tool to find the current time in Lagos, Africa.
Plan: Use the get_current_time tool to get the current time for Lagos in Africa.
Results: The current time in Lagos, [Africa] is date: 12/29/2024 and time: 21:52

Query: What's the population of Nigeria?
Response: As of 2023, the estimated population of Nigeria is approximately 223 million people. This figure can vary
based on different sources and the timing of the estimates.

Query: What is the time in New York (America) right now?
Response: Thought: I need to use the time retrieval tool to get the current time in New York, America.
Plan: Use the get_current_time tool to get the current time in New York, America.. Return the result of the current
time.
Results: The current time in New York, [America] is date: 12/29/2024 and time: 15:52

Query: Subtract 100 from 1000
Response: 1000 - 100 = 900